In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal, invgamma

np.random.seed(200)

# Nombre d'observations
nombre_observations = 50

# Matrice de modèle simulée
matrice_modele = np.column_stack([np.ones(nombre_observations), np.random.normal(0, 1, nombre_observations),
                                  np.random.normal(5, 10, nombre_observations), np.random.normal(100, 10, nombre_observations)])

# Vrais coefficients beta
vrais_coefficients_beta = np.array([1000, 50, -50, 10])

# Vraie valeur de phi
vraie_phi = 10000
matrice_identite = np.eye(nombre_observations)  # Matrice identité utilisée pour la matrice de covariance

# Simuler la variable dépendante pour la régression
variable_dependante = multivariate_normal.rvs(mean=np.dot(matrice_modele, vrais_coefficients_beta),
                                              cov=vraie_phi * matrice_identite)

# Simuler de nombreuses variables dépendantes... utilisées plus tard pour les évaluations asymptotiques
variables_dependantes_liste = np.array([multivariate_normal.rvs(mean=np.dot(matrice_modele, vrais_coefficients_beta),
                                                                 cov=vraie_phi * matrice_identite) for _ in range(1000)])

# Fonction pour l'échantillonneur de Gibbs bloqué
def echantillonneur_gibbs_bloc(y, x, iterations, brulages, rognages):
    # Initialisation de l'échantillonneur de Gibbs
    matrice_x_transposee_x_inverse = np.linalg.inv(np.dot(x.T, x))  # Calculé une fois pour une utilisation répétée dans l'échantillonneur
    phi = np.zeros(iterations)  # Coquille pour phi
    coefficients_beta = np.zeros((iterations, x.shape[1]))  # Coquille pour les coefficients beta
    phi[0] = 6  """ A changer ? """ # Valeur initiale de phi pour démarrer l'échantillonneur   

    # Hyperparamètres de phi
    zeta = 0.5               """ A changer ? """
    eta = 1000                """ A changer ? """

    # Échantillonnage de Gibbs
    for i in range(1, iterations):
        coefficients_beta[i, :] = multivariate_normal.rvs(
            mean=np.dot(np.dot(matrice_x_transposee_x_inverse, x.T), y),
            cov=phi[i - 1] * matrice_x_transposee_x_inverse
        )
        phi[i] = invgamma.rvs(
            a=y.size / 2 + zeta,
            scale=0.5 * np.dot((y - np.dot(x, coefficients_beta[i, :])).T, (y - np.dot(x, coefficients_beta[i, :]))) + eta
        )           # a ou peut etre zeta

    # Application du brûlage et du rognage
    indices_garder = np.arange(brulages, iterations, rognages)
    phi = phi[indices_garder]
    coefficients_beta = coefficients_beta[indices_garder, :]

    # Formater et produire
    resultat = pd.DataFrame(coefficients_beta, columns=['Coefficient_Beta_' + str(i) for i in range(x.shape[1])])
    resultat['Phi'] = phi
    resultat['Iteration'] = indices_garder

    return resultat

# Exécuter l'échantillonneur de Gibbs avec des paramètres spécifiés
resultat_echantillonneur = echantillonneur_gibbs_bloc(y=variable_dependante, x=matrice_modele, iterations=500000,
                                                       brulages=100000, rognages=50)
